In [2]:
import pickle

import pandas as pd
pd.set_option("display.max_columns",999)
#pd.set_option("display.max_rows",5000)

import numpy as np

## Load Data

In [3]:
with open("Spieltagsdaten_Bundesliga_de_raw.pickle", "rb") as handle:
    df_alle = pickle.load(handle)

## Set np.nans were necessary

In [4]:
cols_nans = df_alle.columns[df_alle.isnull().any()]
for col in cols_nans:
    df_alle.loc[df_alle[col].isnull(),col] = np.nan

## Unify teamnames (derzeit nur SV Werder Bremen / Werder Bremen

In [4]:
df_alle.Teamname = df_alle.Teamname.str.replace("SV Werder Bremen","Werder Bremen")

## Rename and rearrange columns 

In [5]:
df_alle = df_alle.reset_index()
df_alle.rename(columns={'index':'ort'}, inplace=True)
new_cols = df_alle.columns.str.replace("tracking","")
new_cols = [el.lower() for el in new_cols]
df_alle.columns = new_cols
#df_alle.head(2)
#list(zip(np.arange(0,len(new_cols)),new_cols)) # to check columns and position

In [6]:
neworder = [7,3,4,5,2,0,6,1] + list(np.arange(8,len(new_cols),1))
sorted_cols = [new_cols[el] for el in neworder] 
df_alle = df_alle[sorted_cols]

## Insert St. Pauli vs. FC Schalke (am grünen Tisch entschieden) - 2010/2011 28. Spieltag

In [7]:
insert_pauli = pd.DataFrame(["28. Spieltag, FC St. Pauli : FC Schalke 04 (grüner Tisch)", "2010/2011",9,28,"01.04.2011 - 20:30","Home","FC St. Pauli","Holger Stanislawski",0]).T
insert_schalke = pd.DataFrame(["28. Spieltag, FC St. Pauli : FC Schalke 04 (grüner Tisch)", "2010/2011",9,28,"01.04.2011 - 20:30","Away","FC Schalke 04","Ralf Rangnick",2]).T

insert_df = pd.concat([insert_pauli, insert_schalke], axis = 0)
insert_df.columns = ["titel","saison","spiel","spieltag","datum","ort","teamname","coach","tore"]
insert_df.reset_index(drop = True, inplace = True)

columns = df_alle.columns[9::]
nan_df = pd.DataFrame(np.zeros(2 * len(columns)).reshape(2,len(columns)),columns = columns)
nan_df.loc[:,:] = np.nan

insert_data = pd.concat([insert_df,nan_df], axis = 1)

df_alle=pd.concat([df_alle,insert_data],axis = 0)
#df_alle.tail()

In [8]:
df_alle = df_alle.sort_values(by = ["saison","spieltag","spiel"], ascending = [True,True,True]).reset_index(drop = True)

## Change data format for columns

In [9]:
## formatierte Datetime
df_alle.datum = pd.to_datetime(df_alle.datum, dayfirst=True)

## formatiere Integer
#int_cols = ["spiel","tore","sprints", "yellowredcards","shotstotalinsidebox","crossesright","shotstotalheader",
#            "yellowcards","fastruns","offsides","intensiveruns","foulscommitted","shotstotal","ballstouched","cornerkicks","redcards",
#            "shotsfootinsidebox","activeplayercount","duelswon","crossesleft","cornerkicksright","passescompleted","crosses","cornerkicksleft",
#            "shotstotaloutsidebox","handballs","passesfailed","shotsfootoutsidebox"]

#for col in int_cols:
#    df_alle[col] = df_alle[col].astype(int)

## formatiere Category
cat_cols = ["saison","ort","teamname","coach"]
for col in cat_cols:
    df_alle[col] = df_alle[col].astype("category")

## formatiere Float
not_floats = ["titel"]
float_cols = [col for col in df_alle.columns if (df_alle[col].dtypes == "object") & (col not in not_floats)]
            
for col in float_cols:
    df_alle[col] = df_alle[col].astype(float)


## Erstelle Indexnummer je Spiel

In [10]:
maxnr = int(len(df_alle)/2)
gamenumber = np.linspace(1,maxnr,maxnr)

df_alle.loc[df_alle.ort == "Home","spielnr"] = gamenumber
df_alle.loc[df_alle.ort == "Away","spielnr"] = gamenumber

## Bestimme Spielausgang (1 = Heimsieg, 0 = Unentschieden, 2 = Auswärtssieg)

In [11]:
home = df_alle[df_alle.ort == "Home"].tore.reset_index(drop = True)
away = df_alle[df_alle.ort == "Away"].tore.reset_index(drop = True)

In [12]:
home_res = np.where(home - away > 0, 1,np.where(home-away == 0,0,2))
away_res = np.where(away - home > 0,2,np.where(away-home == 0,0,1))

In [13]:
df_alle.loc[df_alle.ort == "Home","spielausgang"] = home_res
df_alle.loc[df_alle.ort == "Away","spielausgang"] = away_res

## Bestimme Ausgangstyp ("Heimsieg", "Heimniederlage", "Unentschieden", "Auswärtssieg", "Auswärtsniederlage")

In [14]:
df_alle.loc[(df_alle.ort == "Home") & (df_alle.spielausgang == 1), "ausgangstyp"] = "Heimsieg"
df_alle.loc[(df_alle.ort == "Home") & (df_alle.spielausgang == 2), "ausgangstyp"] = "Heimniederlage"
df_alle.loc[(df_alle.ort == "Away") & (df_alle.spielausgang == 1), "ausgangstyp"] = "Auswärtsniederlage"
df_alle.loc[(df_alle.ort == "Away") & (df_alle.spielausgang == 2), "ausgangstyp"] = "Auswärtssieg"
df_alle.loc[df_alle.spielausgang == 0,"ausgangstyp"] = "Unentschieden"
df_alle.spielausgang = df_alle.spielausgang.astype("category")

## Bestimme Punkte und die Tordifferenz

In [15]:
## Punkte
mapper = {"Heimsieg":3,"Heimniederlage":0,"Auswärtssieg":3,"Auswärtsniederlage":0,"Unentschieden":1}
df_alle["punkte"] = df_alle.ausgangstyp.map(mapper)

In [16]:
## Tordifferenz
home_tore = np.array(df_alle[df_alle.ort == "Home"]["tore"])
away_tore = np.array(df_alle[df_alle.ort == "Away"]["tore"])
df_alle.loc[df_alle.ort == "Home","tordiff"] = home_tore - away_tore
df_alle.loc[df_alle.ort == "Away","tordiff"] = (home_tore - away_tore)*-1

## Berechne Tabellen-Daten (Platzierung, Punkte, Tordiff etc. am jeweiligen Spieltag)

In [17]:
mapper = {"2009/2010":2009,"2010/2011":2010,"2011/2012":2011,"2012/2013":2012,"2013/2014":2013,
          "2014/2015":2014,"2015/2016":2015,"2016/2017":2016,"2017:2018":2017}
df_alle.saison = df_alle.saison.map(mapper)

In [18]:
def get_spieltagstabelle(df, saison,spieltag):
    
    '''Berechnet Spieltagstabelle für angegebenen Spieltag einer Saison
    ::input: df = KickerDaten, saison (e.g. 2015/2016 = 2015), spieltag (e.g. 12. Spieltag = 12)
    ::output: pandas mit Spieltagstabelle
    
    Beispiel: get_spieltagstabelle(df, 2015, 12)
    '''
    if (saison not in df.saison.unique()) or (spieltag not in df[df.saison == saison].spieltag.unique()): 
        print("Saison und/oder Spieltag der Saison nicht vorhanden") 
        
        return 
    
    punkte = df.groupby(["saison","spieltag","teamname"])["punkte"].sum().unstack(1).cumsum(axis = 1)
    tore = df.groupby(["saison","spieltag","teamname"])["tore"].sum().unstack(1).cumsum(axis = 1)
    tordiff = df.groupby(["saison","spieltag","teamname"])["tordiff"].sum().unstack(1).cumsum(axis = 1)
    gegentore = tore - tordiff

    df_punkte = punkte.loc[saison, spieltag].reset_index()
    df_punkte.columns = ["teamname","saison_punkte"]

    df_tore = tore.loc[saison,spieltag].reset_index()
    df_tore.columns = ["teamname","saison_tore"]

    df_gegentore = gegentore.loc[saison,spieltag].reset_index()
    df_gegentore.columns = ["teamname","saison_gegentore"]

    tabelle_unsorted = df_punkte.merge(df_tore, on = "teamname").merge(df_gegentore, on = "teamname")
    tabelle_unsorted["saison_tordiff"] = tabelle_unsorted["saison_tore"]-tabelle_unsorted["saison_gegentore"]
    
    spieltagstabelle = tabelle_unsorted.sort_values(by = ["saison_punkte","saison_tordiff","saison_tore","saison_gegentore"], ascending = False).set_index(np.arange(1,19))
    spieltagstabelle["saison_platz"] = spieltagstabelle.index
    spieltagstabelle["saison_gegentore"] = spieltagstabelle["saison_gegentore"].astype(float)
    spieltagstabelle["saison_tordiff"] = spieltagstabelle["saison_tordiff"].astype(float)
    spieltagstabelle["saison"] = saison
    spieltagstabelle["spieltag"] = spieltag
    
    return spieltagstabelle


In [19]:
## generiere Pandas mit allen Tabellen für alle Saisons und Spieltage
saisons = df_alle.saison.unique()
spieltage = df_alle.spieltag.unique()
all_spieltagstabellen = pd.DataFrame()
for saison in saisons:
    for spieltag in spieltage:
        tabelle = get_spieltagstabelle(df_alle, saison,spieltag)
        all_spieltagstabellen = pd.concat([all_spieltagstabellen, tabelle],axis=0)

## Merge Daten aus Tabellen (Punkte, Tore, Platzierung etc.)
df_alle = df_alle.merge(all_spieltagstabellen, on = ["teamname","saison","spieltag"], how = "outer")

In [20]:
df_alle.columns, len(df_alle.columns)

(Index(['titel', 'saison', 'spiel', 'spieltag', 'datum', 'ort', 'teamname',
        'coach', 'tore', 'averagespeed', 'sprints', 'ballstouchedpercent',
        'yellowredcards', 'shotstotalinsidebox', 'shotsongoaltotal',
        'passescompletedpercent', 'crossesright', 'shotstotalheader',
        'yellowcards', 'fastruns', 'offsides', 'intensiveruns',
        'foulscommitted', 'fastrunsdistance', 'shotstotal', 'ballstouched',
        'passesfailedpercent', 'cornerkicks', 'redcards', 'shotsfootinsidebox',
        'duelswonpercent', 'activeplayercount', 'duelswon', 'crossesleft',
        'cornerkicksright', 'speed', 'passescompleted', 'intensiverunsdistance',
        'crosses', 'cornerkicksleft', 'shotstotaloutsidebox', 'handballs',
        'distance', 'passesfailed', 'shotsfootoutsidebox', 'sprintsdistance',
        'spielnr', 'spielausgang', 'ausgangstyp', 'punkte', 'tordiff',
        'saison_punkte', 'saison_tore', 'saison_gegentore', 'saison_tordiff',
        'saison_platz'],
       

## Füge Chancenverwertungs-KPIs hinzu

In [21]:
df_alle["chancenverwertung"] = df_alle.tore / df_alle.shotstotal

In [22]:
df_alle["chancen_inside"] = df_alle.shotstotalinsidebox / df_alle.shotstotal

## Sortiere Spalten final

In [23]:
columns_main = ["titel","saison","spiel","spieltag","spielnr","datum","ort","teamname","coach","tore","tordiff","punkte","spielausgang","ausgangstyp"]
columns_shots = ["shotstotal","shotsongoaltotal","shotstotaloutsidebox","shotstotalinsidebox","shotsfootoutsidebox","shotsfootinsidebox","shotstotalheader"]
columns_chancen = ["chancenverwertung","chancen_inside"]
columns_passes = ["passescompletedpercent","passesfailedpercent","passescompleted","passesfailed"]
columns_corners = ["cornerkicks","cornerkicksleft","cornerkicksright"]
columns_crosses = ["crosses","crossesleft","crossesright"]
columns_runs = ["fastruns","intensiveruns","fastrunsdistance","intensiverunsdistance","averagespeed","speed","distance","sprints","sprintsdistance"]
columns_cards = ["yellowcards","redcards","yellowredcards"]
columns_others = ["offsides","foulscommitted","ballstouchedpercent","ballstouched","handballs","duelswonpercent","duelswon",
                  "activeplayercount"]
columns_saison = ["saison_platz","saison_punkte","saison_tordiff","saison_tore","saison_gegentore"]


columns_sorted = columns_main + columns_shots + columns_chancen + columns_passes + columns_corners + columns_crosses + columns_runs + columns_cards + columns_others + columns_saison
df_alle = df_alle[columns_sorted]

## Save Data

In [24]:
with open('Spieltagsdaten_Bundesliga_de_formatted.pickle', 'wb') as handle:
    pickle.dump(df_alle, handle, protocol=pickle.HIGHEST_PROTOCOL)